In [1]:
import pandas as pd

In [2]:
# Función que genera nuevos indices numericos, desde 0 hasta n-1.
# n: Númmero total de filas del DataFrame
def nuevosIndices(tabla):
    tabla.index=[i for i in range(len(tabla))]
    return tabla

def depurarColumnaIndices(tabla):
    nombrePrimeraCol=tabla.columns[0]
    primeraColumna=tabla[nombrePrimeraCol]
    indices=tabla.index
    if all([indices[i]==primeraColumna[i] for i in indices]):
        tabla=tabla.drop([nombrePrimeraCol],axis=1)
    return tabla

# Función que elimina filas, teniendo en cuenta una columna de valores numéricos.
# Se busca depurar las filas que contengan datos atípicos.
def depurarNumeros(tabla, nombreCol, limInf, limSup):
    indices=tabla.index
    for i in indices:
        fila=tabla.loc[i]
        if fila[nombreCol]<limInf or fila[nombreCol]>limSup:
            tabla=tabla.drop([i],axis=0)
    tabla=nuevosIndices(tabla)
    return tabla

# Función que elimina columnas que tengan un porcentaje de valores indefinidos (NaN) demasiado grande para el problema a tratar.
def depurarColumnas(tabla, porcentaje):
    size=len(tabla)
    columnas=tabla.columns
    for columna in columnas:
        booleanos=tabla[columna].notnull()
        if sum(booleanos)<size*porcentaje:
            tabla=tabla.drop([columna],axis=1)
    tabla=nuevosIndices(tabla)
    return tabla

# Función que da información detallada de cada columna del DataFrame.
# Usar con DataFrames grandes.
def describirColumnas(tabla, factor):
    size=len(tabla)
    columnas=tabla.columns
    for i in range(len(columnas)//factor+1):
        subgrupo=columnas[0+i*factor:factor+i*factor]
        print('*'*85+'SUBGRUPO #'+str(i+1)+'*'*85+'\n')
        print(datos[subgrupo].describe(include='all'))
        input('\nPresione la tecla Enter para mostrar el siguiente subgrupo.\n')

#Función que elimina grupos de columnas innecesarias.
def borrarColumnasInnecesarias(tabla,columnas):
    for columna in columnas:
        tabla=tabla.drop([columna],axis=1)
    return tabla

#Función que desplaza los elementos de una fila hacia la izquierda, entre cierto rango de columnas.
def moverHaciaIzquierda(tabla, posBooleanas, nombreColumna='Unnamed: 89', inicio=6, fin=None):
    columnas=tabla.columns
    if fin is None:
        fin=len(columnas)
    columnas=columnas[inicio:fin]
    for j in range(len(posBooleanas)):
        if posBooleanas[j]:
            aux=tabla.loc[j, columnas[0]]
            for i in range(len(columnas)-1):
                tabla.loc[j, columnas[i]]=tabla.loc[j, columnas[i+1]]
            tabla.loc[j, columnas[i+1]]=aux
    return tabla

In [10]:
#Columnas para borrar sin problema: 'Unnamed: 0', 'ID', 'Photo', 'Flag', 'Club Logo', 'Unnamed: 89'
#Nota: La última columna hay que revisarla para mover unos datos.
datos=pd.read_csv('fifa.csv')
columnasParaBorrar=['Unnamed: 0','ID','Photo', 'Flag', 'Club Logo']
datos=borrarColumnasInnecesarias(datos, columnasParaBorrar)

In [11]:
#Se soluciona el problema de los datos desplazados y se elimina la columna 'Unnamed: 89'
datos=moverHaciaIzquierda(datos, datos['Unnamed: 89'].notnull())
datos=datos.drop(['Unnamed: 89'],axis=1)

In [13]:
#Se toman las edades entre 15 y 100 como datos válidos y se halla un promedio de estos datos.
#Posteriormente se reemplaza en aquellas filas con datos atípicos.
subconjunto=datos[(datos['Age']>15) & (datos['Age']<100)] #Es necesario usar el ampersand y símbolos similares para realizar operaciones lógicas.
promEdades=int(subconjunto['Age'].mean()//1)

#.loc y .iloc permiten realizar el cambio de un valor dado un condicional o ubicaciones. 
#.replace se usa para cambiar valores específicos (el método se encarga de buscar sus posiciones).
datos.loc[(datos['Age']<15) | (datos['Age']>100),'Age']=promEdades

In [ ]:
#Mostrar la cantidad de datos NaN que hay en cada columna
#Columnas sin valores NaN: Unnamed: 0, ID, Name, Age, Photo, Flag, Overall, Potential, Value, Wage, Special
columnas=datos.columns
setColumnas=columnas[80:90]
print(datos[setColumnas].isnull().sum())

In [ ]:
#Muestra el tipo de datos presente en cada columna de un grupo determinado de columnas.
columnas=datos.columns
setColumnas=columnas[0:10]
print(datos[setColumnas].dtypes)

In [17]:
#Guardar Dataframe depurado.
datos.to_csv('nuevoFifa.csv')
datos.to_excel('nuevoFifa.xlsx')